En la presente etapa del estudio se busca determinar los hiperparámetros del modelo. Una vez que se tenga aquella combinación con menor pérdida, se pasa a estimar el modelo en una segunda etapa.

In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
parentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentdir)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/D2FM')
module_path = '/content/drive/My Drive/D2FM'
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
!pip install pykalman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
from models.ddfm import *

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Tuple
from tensorflow import keras
from tensorflow.keras import layers
from statsmodels.tsa.statespace.dynamic_factor_mq import DynamicFactorMQ as DFM
from sklearn.model_selection import ParameterGrid
from typing import Tuple
import numpy as np
from pykalman import KalmanFilter
import json
import time

In [ ]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import random
from sklearn.metrics import mean_squared_error as mse
from typing import Tuple
from sklearn.preprocessing import StandardScaler

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/D2FM/scaled_train.csv')
df_test=pd.read_csv('/content/drive/MyDrive/D2FM/scaled_test.csv')


In [ ]:
df_train_idx=df_train.set_index("Index")
df_test_idx=df_test.set_index("Index")

# K=1 para determinar los hiperparámetros del modelo. Son X combinaciones en una sola validación. Se determina el "mejor" conjunto de hiperparámetros en base a la pérdida menor.

In [ ]:
# Outer loop for r values
for r in [5]:
    start_time_total = time.time()  # Record the start time for the entire process

    structure_encoders = [(r * 12, r * 6, r * 4, r), (r * 24, r * 12, r * 8, r)]

    lags_inputs = list(range(2))
    link_options = ['tanh', 'relu']
    batch_size_options = [72, 100]

    model_info = []

    # Iterate over both True and False values for use_bias
    for use_bias_setting in [True, False]:
        # Iterate over each structure_encoder configuration
        for structure_encoder in structure_encoders:
            # Iterate over each lag value
            for lags_input in lags_inputs:
                # Iterate over each link option
                for link_option in link_options:
                    # Iterate over each batch size option
                    for batch_size_option in batch_size_options:
                        # Initialize the DDFM with the current configuration and use_bias setting
                        ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                                    optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link=link_option,
                                    batch_size=batch_size_option, epochs=100, max_iter=1000)

                        # Start the timer for each individual model
                        start_time_model = time.time()

                        # Fit the model
                        ddfm.fit()

                        # Stop the timer for each individual model
                        end_time_model = time.time()

                        # Calculate the execution time for each individual model
                        execution_time_model = end_time_model - start_time_model

                        # Store the model, its configuration, final loss, and execution time
                        final_loss = ddfm.loss_now
                        # Store the model, its configuration, final loss, execution time, and r value
                        model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss,
                              execution_time_model, link_option, batch_size_option, r))


    # Calculate the total execution time
    end_time_total = time.time()
    total_execution_time = end_time_total - start_time_total

    # Save the list of dictionaries to a JSON file
    model_info_dict_list = []
    for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model, link_option, batch_size_option , r_value in model_info:
        model_info_dict = {
            "encoder_config": encoder_config,
            "lag": lag,
            "use_bias_setting": use_bias_setting,
            "final_loss": final_loss,
            "execution_time_model": execution_time_model,
            "link_option": link_option,
            "batch_size_option": batch_size_option,
            "r_value": r_value
        }
        model_info_dict_list.append(model_info_dict)

    # Save the list of dictionaries to a JSON file
    with open(f"/content/drive/MyDrive/D2FM/model_info_r5.json", "w") as json_file:
        json.dump(model_info_dict_list, json_file)

    print(f"Results for r={r} saved to model_info_r{r}.json")
    print(f"Total Execution Time for all models for r={r}: {total_execution_time:.2f} seconds")
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time, link_option, batch_size_option, r_value in model_info:
    # Update min_loss and min_loss_config if current loss is smaller
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option, r_value)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option , r_value= min_loss_config
    print(f"Minimum Loss: {min_loss} with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option}, r {r_value}")
else:
    print("No models found or no valid loss values.")

@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 2 iterations - new loss: 0.6337849734197494 - delta: 0.00012342317393720145 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.6299177772556381 - delta: 0.00011907978845490698 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 3 iterations - new loss: 0.644579046094442 - delta: 0.00034606306938838287 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6285120588164306 - delta: 0.0002369827365331642 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.6224360048604499 - delta: 0.0003318787530264389 < 0.0005
@Info - Note: Sort

In [ ]:
# Outer loop for r values
for r in [6]:
    start_time_total = time.time()  # Record the start time for the entire process

    structure_encoders = [(r * 12, r * 6, r * 4, r), (r * 24, r * 12, r * 8, r)]

    lags_inputs = list(range(2))
    link_options = ['tanh', 'relu']
    batch_size_options = [72, 100]

    model_info = []

    # Iterate over both True and False values for use_bias
    for use_bias_setting in [True, False]:
        # Iterate over each structure_encoder configuration
        for structure_encoder in structure_encoders:
            # Iterate over each lag value
            for lags_input in lags_inputs:
                # Iterate over each link option
                for link_option in link_options:
                    # Iterate over each batch size option
                    for batch_size_option in batch_size_options:
                        # Initialize the DDFM with the current configuration and use_bias setting
                        ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                                    optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link=link_option,
                                    batch_size=batch_size_option, epochs=100, max_iter=1000)

                        # Start the timer for each individual model
                        start_time_model = time.time()

                        # Fit the model
                        ddfm.fit()

                        # Stop the timer for each individual model
                        end_time_model = time.time()

                        # Calculate the execution time for each individual model
                        execution_time_model = end_time_model - start_time_model

                        # Store the model, its configuration, final loss, and execution time
                        final_loss = ddfm.loss_now
                        # Store the model, its configuration, final loss, execution time, and r value
                        model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss,
                              execution_time_model, link_option, batch_size_option, r))


    # Calculate the total execution time
    end_time_total = time.time()
    total_execution_time = end_time_total - start_time_total

    # Save the list of dictionaries to a JSON file
    model_info_dict_list = []
    for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model, link_option, batch_size_option , r_value in model_info:
        model_info_dict = {
            "encoder_config": encoder_config,
            "lag": lag,
            "use_bias_setting": use_bias_setting,
            "final_loss": final_loss,
            "execution_time_model": execution_time_model,
            "link_option": link_option,
            "batch_size_option": batch_size_option,
            "r_value": r_value
        }
        model_info_dict_list.append(model_info_dict)

    # Save the list of dictionaries to a JSON file
    with open(f"/content/drive/MyDrive/D2FM/model_info_r5.json", "w") as json_file:
        json.dump(model_info_dict_list, json_file)

    print(f"Results for r={r} saved to model_info_r{r}.json")
    print(f"Total Execution Time for all models for r={r}: {total_execution_time:.2f} seconds")
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time, link_option, batch_size_option, r_value in model_info:
    # Update min_loss and min_loss_config if current loss is smaller
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option, r_value)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option , r_value= min_loss_config
    print(f"Minimum Loss: {min_loss} with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option}, r {r_value}")
else:
    print("No models found or no valid loss values.")

@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 9 iterations - new loss: 0.6034720033485297 - delta: 0.00023677804003644545 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5972880867586865 - delta: 0.00046984519310047976 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.6055026109538978 - delta: 6.382738273784761e-05 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.598249991370626 - delta: 1.1014098662169128e-05 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 3ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.590212278963928 - delta: 0.00046660506371375295 < 0.0005
@Info - Note: Sort

In [ ]:
# Outer loop for r values
for r in [7]:
    start_time_total = time.time()  # Record the start time for the entire process

    structure_encoders = [(r * 12, r * 6, r * 4, r), (r * 24, r * 12, r * 8, r)]

    lags_inputs = list(range(2))
    link_options = ['tanh', 'relu']
    batch_size_options = [72, 100]

    model_info = []

    # Iterate over both True and False values for use_bias
    for use_bias_setting in [True, False]:
        # Iterate over each structure_encoder configuration
        for structure_encoder in structure_encoders:
            # Iterate over each lag value
            for lags_input in lags_inputs:
                # Iterate over each link option
                for link_option in link_options:
                    # Iterate over each batch size option
                    for batch_size_option in batch_size_options:
                        # Initialize the DDFM with the current configuration and use_bias setting
                        ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                                    optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link=link_option,
                                    batch_size=batch_size_option, epochs=100, max_iter=1000)

                        # Start the timer for each individual model
                        start_time_model = time.time()

                        # Fit the model
                        ddfm.fit()

                        # Stop the timer for each individual model
                        end_time_model = time.time()

                        # Calculate the execution time for each individual model
                        execution_time_model = end_time_model - start_time_model

                        # Store the model, its configuration, final loss, and execution time
                        final_loss = ddfm.loss_now
                        # Store the model, its configuration, final loss, execution time, and r value
                        model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss,
                              execution_time_model, link_option, batch_size_option, r))


    # Calculate the total execution time
    end_time_total = time.time()
    total_execution_time = end_time_total - start_time_total

    # Save the list of dictionaries to a JSON file
    model_info_dict_list = []
    for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model, link_option, batch_size_option , r_value in model_info:
        model_info_dict = {
            "encoder_config": encoder_config,
            "lag": lag,
            "use_bias_setting": use_bias_setting,
            "final_loss": final_loss,
            "execution_time_model": execution_time_model,
            "link_option": link_option,
            "batch_size_option": batch_size_option,
            "r_value": r_value
        }
        model_info_dict_list.append(model_info_dict)

    # Save the list of dictionaries to a JSON file
    with open(f"/content/drive/MyDrive/D2FM/model_info_r5.json", "w") as json_file:
        json.dump(model_info_dict_list, json_file)

    print(f"Results for r={r} saved to model_info_r{r}.json")
    print(f"Total Execution Time for all models for r={r}: {total_execution_time:.2f} seconds")
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time, link_option, batch_size_option, r_value in model_info:
    # Update min_loss and min_loss_config if current loss is smaller
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option, r_value)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option , r_value= min_loss_config
    print(f"Minimum Loss: {min_loss} with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option}, r {r_value}")
else:
    print("No models found or no valid loss values.")

@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 5 iterations - new loss: 0.5727474687587538 - delta: 0.0003824681869214333 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 8 iterations - new loss: 0.5659209500689458 - delta: 0.00011515628523569736 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 4ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5807477376922936 - delta: 0.0003383810926425854 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 9 iterations - new loss: 0.5682630322570074 - delta: 0.0003281696843942158 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5603617106899877 - delta: 0.00021160238970096345 < 0.0005
@Info - Note: Sort

Resultados asociados a la minima Loss:

* Minimum Loss: 0.6215284234293389 with encoder (120, 60, 40, 5), lag 1, use_bias False, link tanh, batch_size 72, r 5


* Minimum Loss: 0.589956869321811 with encoder (144, 72, 48, 6), lag 1, use_bias True, link tanh, batch_size 100, r 6

* Minimum Loss: 0.560155765006405 with encoder (168, 84, 56, 7), lag 1, use_bias True, link tanh, batch_size 72, r 7

In [ ]:
######################## no borrar ##################
# comparacion con los 3 factores
######################################################
import time

# Outer loop for r values
for r in [5, 6, 7]:
    start_time_total = time.time()  # Record the start time for the entire process

    structure_encoders = [(r * 12, r * 6, r * 4, r), (r * 24, r * 12, r * 8, r)]

    lags_inputs = list(range(2))
    link_options = ['tanh', 'relu']
    batch_size_options = [72, 100]

    model_info = []

    # Iterate over both True and False values for use_bias
    for use_bias_setting in [True, False]:
        # Iterate over each structure_encoder configuration
        for structure_encoder in structure_encoders:
            # Iterate over each lag value
            for lags_input in lags_inputs:
                # Iterate over each link option
                for link_option in link_options:
                    # Iterate over each batch size option
                    for batch_size_option in batch_size_options:
                        # Initialize the DDFM with the current configuration and use_bias setting
                        ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                                    optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link=link_option,
                                    batch_size=batch_size_option, epochs=100, max_iter=1000)

                        # Start the timer for each individual model
                        start_time_model = time.time()

                        # Fit the model
                        ddfm.fit()

                        # Stop the timer for each individual model
                        end_time_model = time.time()

                        # Calculate the execution time for each individual model
                        execution_time_model = end_time_model - start_time_model

                        # Store the model, its configuration, final loss, and execution time
                        final_loss = ddfm.loss_now
                        # Store the model, its configuration, final loss, execution time, and r value
                        model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss,
                              execution_time_model, link_option, batch_size_option, r))


    # Calculate the total execution time
    end_time_total = time.time()
    total_execution_time = end_time_total - start_time_total

    # Print the information for each model
    for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model, link_option, batch_size_option , r_value in model_info:
        print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option}, r {r_value} - Final Loss: {final_loss}")
        print(f"Execution Time for this model: {execution_time_model:.2f} seconds")

    print(f"Total Execution Time for all models: {total_execution_time:.2f} seconds")

@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 3 iterations - new loss: 0.6339407159860809 - delta: 9.398676472979998e-05 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.6289796917417518 - delta: 0.00048051832003069724 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 5 iterations - new loss: 0.6399867368474991 - delta: 1.1401713881717915e-05 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 3 iterations - new loss: 0.6295021626708045 - delta: 0.00017737622706977927 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.6222877991209359 - delta: 0.0003756796052888426 < 0.0005
@Info - Note: Sor

In [ ]:
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time, link_option, batch_size_option, r_value in model_info:
    # Update min_loss and min_loss_config if current loss is smaller
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option, r_value)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option , r_value= min_loss_config
    print(f"Minimum Loss: {min_loss} with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option}, r {r_value}")
else:
    print("No models found or no valid loss values.")

Minimum Loss: 0.5600313739508542 with encoder (168, 84, 56, 7), lag 1, use_bias False, link tanh, batch_size 100, r 7


In [ ]:
import json

# Create a list to store dictionaries representing each model
model_info_dict_list = []

# Iterate over each model's information and convert it into a dictionary
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model, link_option, batch_size_option, r_value in model_info:
    model_info_dict = {
        "encoder_config": encoder_config,
        "lag": lag,
        "use_bias_setting": use_bias_setting,
        "final_loss": final_loss,
        "execution_time_model": execution_time_model,
        "link_option": link_option,
        "batch_size_option": batch_size_option,
        "r_value": r_value
    }
    model_info_dict_list.append(model_info_dict)

# Save the list of dictionaries to a JSON file
with open("/content/drive/MyDrive/D2FM/model_info.json", "w") as json_file:
    json.dump(model_info_dict_list, json_file)

print("Model information saved to model_info.json")

Model information saved to model_info.json


In [ ]:
# Dictionary to store minimum loss and associated configuration for each r value
min_loss_configs = {}

# Iterate over unique r values
for r_value in set(r for _, _, _, _, _, _, _, _, r in model_info):
    # Initialize variables to store the minimum loss and associated configuration for current r value
    min_loss = float('inf')
    min_loss_config = None

    # Iterate over the stored models to find the one with the minimum loss for current r value
    for model, encoder_config, lag, use_bias_setting, final_loss, execution_time, link_option, batch_size_option, r in model_info:
        # Check if current model corresponds to current r value
        if r == r_value:
            # Update min_loss and min_loss_config if current loss is smaller
            if final_loss < min_loss:
                min_loss = final_loss
                min_loss_config = (encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option, r)

    # Store the minimum loss and associated configuration for current r value in the dictionary
    min_loss_configs[r_value] = (min_loss, min_loss_config)

# Print the configurations with the minimum loss for each r value
for r_value in sorted(min_loss_configs.keys()):
    min_loss, min_loss_config = min_loss_configs[r_value]
    if min_loss_config is not None:
        encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option, _ = min_loss_config
        print(f"Minimum Loss for r={r_value}: {min_loss} with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option}")
    else:
        print(f"No models found or no valid loss values for r={r_value}.")




Minimum Loss for r=7: 0.5600313739508542 with encoder (168, 84, 56, 7), lag 1, use_bias False, link tanh, batch_size 100


In [ ]:
import time

start_time_total = time.time()  # Record the start time for the entire process
r = 6
structure_encoders = [(r * 6, r * 4, r * 2, r),
                      (r * 12, r * 6, r * 4, r), (r * 24, r * 12, r * 8, r), (r * 48, r * 24, r * 16, r)]

lags_inputs = list(range(2))
link_options = ['tanh', 'relu']
batch_size_options = [24, 100]

model_info = []

# Iterate over both True and False values for use_bias
for use_bias_setting in [True, False]:
    # Iterate over each structure_encoder configuration
    for structure_encoder in structure_encoders:
        # Iterate over each lag value
        for lags_input in lags_inputs:
            # Iterate over each link option
            for link_option in link_options:
                # Iterate over each batch size option
                for batch_size_option in batch_size_options:
                    # Initialize the DDFM with the current configuration and use_bias setting
                    ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                                optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link=link_option,
                                batch_size=batch_size_option, epochs=100, max_iter=1000)

                    # Start the timer for each individual model
                    start_time_model = time.time()

                    # Fit the model
                    ddfm.fit()

                    # Stop the timer for each individual model
                    end_time_model = time.time()

                    # Calculate the execution time for each individual model
                    execution_time_model = end_time_model - start_time_model

                    # Store the model, its configuration, final loss, and execution time
                    final_loss = ddfm.loss_now
                    model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss,
                                       execution_time_model, link_option, batch_size_option))


# Calculate the total execution time
end_time_total = time.time()
total_execution_time = end_time_total - start_time_total

# Print the information for each model
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model, link_option, batch_size_option in model_info:
    print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option} - Final Loss: {final_loss}")
    print(f"Execution Time for this model: {execution_time_model:.2f} seconds")

print(f"Total Execution Time for all models: {total_execution_time:.2f} seconds")


@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6062719455379042 - delta: 0.0001527789508580558 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6042899097289977 - delta: 0.0003065221216681004 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 2 iterations - new loss: 0.6023843152966347 - delta: 3.685885736727355e-05 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 2 iterations - new loss: 0.598491798566092 - delta: 0.00015911664960780816 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6002428834341017 - delta: 0.00022240091485573735 < 0.0005
@Info - Note: Sorti

In [ ]:
# Initialize variables to store the minimum loss and associated configuration.
min_loss = float('inf')
min_loss_config = None

# Iterate over the stored models to find the one with the minimum loss.
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time, link_option, batch_size_option in model_info:
    # Update min_loss and min_loss_config if current loss is smaller
    if final_loss < min_loss:
        min_loss = final_loss
        min_loss_config = (encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option)

# Print the configuration with the minimum loss.
if min_loss_config is not None:
    encoder_config, lag, use_bias_setting, execution_time, link_option, batch_size_option = min_loss_config
    print(f"Minimum Loss: {min_loss} with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting}, link {link_option}, batch_size {batch_size_option}")
else:
    print("No models found or no valid loss values.")


Minimum Loss: 0.58967100937883 with encoder (288, 144, 96, 6), lag 1, use_bias True, link tanh, batch_size 100


Para menos combinaciones de hiperparámetros, queda como ejemplo:

In [ ]:
import time

start_time_total = time.time()  # Record the start time for the entire process

# Your existing code
r = 6
structure_encoders = [(r * 6, r * 4, r * 2, r),
                      (r * 12, r * 6, r * 4, r), (r * 24, r * 12, r * 8, r), (r * 48, r * 24, r * 16, r),]

lags_inputs = list(range(2))

model_info = []

# Iterate over both True and False values for use_bias
for use_bias_setting in [True, False]:
    # Iterate over each structure_encoder configuration
    for structure_encoder in structure_encoders:
        # Iterate over each lag value
        for lags_input in lags_inputs:
            # Initialize the DDFM with the current configuration and use_bias setting
            ddfm = DDFM(df_train_idx, lags_input=lags_input, structure_encoder=structure_encoder,
                        optimizer='Adam', factor_oder=1, use_bias=use_bias_setting, link='tanh',
                        epochs=100, max_iter=1000)

            # Start the timer for each individual model
            start_time_model = time.time()

            # Fit the model
            ddfm.fit()

            # Stop the timer for each individual model
            end_time_model = time.time()

            # Calculate the execution time for each individual model
            execution_time_model = end_time_model - start_time_model

            # Store the model, its configuration, final loss, and execution time
            final_loss = ddfm.loss_now
            model_info.append((ddfm, structure_encoder, lags_input, use_bias_setting, final_loss, execution_time_model))

# Calculate the total execution time
end_time_total = time.time()
total_execution_time = end_time_total - start_time_total

# Print the information for each model
for model, encoder_config, lag, use_bias_setting, final_loss, execution_time_model in model_info:
    print(f"Model with encoder {encoder_config}, lag {lag}, use_bias {use_bias_setting} - Final Loss: {final_loss}")
    print(f"Execution Time for this model: {execution_time_model:.2f} seconds")

print(f"Total Execution Time for all models: {total_execution_time:.2f} seconds")


@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.6000213739457754 - delta: 2.948570757007814e-05 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 2 iterations - new loss: 0.5981605811324008 - delta: 0.00041992750411991146 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 4 iterations - new loss: 0.6002641262117545 - delta: 0.0003938048813976006 < 0.0005
@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.5901988115578353 - delta: 0.00020263298102667096 < 0.0005
@Info - Note: Sorting data.
19/19 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 7 iterations - new loss: 0.5949236383805527 - delta: 3.758155916742125e-05 < 0.0005
@Info - Note: Sort

In [ ]:
ddfm6 = DDFM(df_train_idx, structure_encoder=(288, 144, 96, 6), lags_input=1, factor_oder=1,
                             use_bias=False, link='tanh', max_iter=1000)
ddfm6.fit()

@Info - Note: Sorting data.
18/18 [==============================] - 0s 2ms/step
@Info: Convergence achieved in 6 iterations - new loss: 0.589643687159019 - delta: 0.00012052289014438879 < 0.0005
